# Summarize escape across all sera

In [21]:
import altair as alt

import pandas as pd

import polyclonal.alphabets

_ = alt.data_transformers.disable_max_rows()

The next cell is tagged as `parameters` for `papermill` parameterization:

In [2]:
site_numbering_map_csv = None
func_effects_csv = None
sera = None

In [3]:
# Parameters
sera = {
    "sera1-01": "results/antibody_escape/averages/sera1-01_mut_escape.csv",
    "sera1-02": "results/antibody_escape/averages/sera1-02_mut_escape.csv",
    "sera1-03": "results/antibody_escape/averages/sera1-03_mut_escape.csv",
    "sera1-04": "results/antibody_escape/averages/sera1-04_mut_escape.csv",
    "sera1-05": "results/antibody_escape/averages/sera1-05_mut_escape.csv",
    "sera2-01": "results/antibody_escape/averages/sera2-01_mut_escape.csv",
    "sera2-04": "results/antibody_escape/averages/sera2-04_mut_escape.csv",
    "sera2-05": "results/antibody_escape/averages/sera2-05_mut_escape.csv",
}
site_numbering_map_csv = "data/site_numbering_map.csv"
func_effects_csv = "results/func_effects/averages/293T_entry_func_effects.csv"

import os
os.chdir("../")

Some configuration for plot:

In [4]:
times_seen = 3  # only include mutations with times_seen >= this
frac_models = 1  # only include mutations in >= this fraction of models / selections
escape_stat = "escape_median"  # for each sera, use this escape value (mean or median)
init_site_escape_stat = "mean"  # default site escape stat to show
init_min_func_effect = -2  # default minimum functional effect to show
init_floor_escape_at_zero = True  # default on whether to floor escape at zero

Read the escape data and add site numbering and functional effect data:

In [10]:
escape_tidy = (
    pd.concat([pd.read_csv(f).assign(serum=s) for s, f in sera.items()])
    .rename(columns={escape_stat: "escape"})
    .query("frac_models >= @frac_models")
    .query("times_seen >= @times_seen")
    [["epitope", "serum", "site", "wildtype", "mutant", "escape"]]
)

assert escape_tidy["epitope"].nunique() == 1, "averaging only works for one epitope"

escape = (
    escape_tidy
    .pivot_table(
        index=["site", "wildtype", "mutant"],
        columns="serum",
        values="escape",
    ).reset_index()
    .assign(site_mutant=lambda x: x["site"].astype(str) + x["mutant"])
)

assert escape["site_mutant"].nunique() == len(escape)

site_numbering_map = (
    pd.read_csv(site_numbering_map_csv)
    .rename(columns={"reference_site": "site"})
    [["site", "sequential_site", "region"]]
)

func_effects = (
    pd.read_csv(func_effects_csv)
    .rename(columns={"effect": "functional effect"})
    .query("times_seen >= @times_seen")
    .assign(frac_selections=lambda x: x["n_selections"] / x["n_selections"].max())
    .query("frac_selections >= @frac_models")
    [["site", "wildtype", "mutant", "functional effect"]]
)
# add wildtype functional effects of zero
func_effects = (
    pd.concat(
        [
            func_effects,
            (
                func_effects
                [["site", "wildtype"]]
                .drop_duplicates()
                .assign(
                    mutant=lambda x: x["wildtype"],
                    **{"functional effect": 0},
                )
            )
        ],
        ignore_index=True,
    )
    .assign(site_mutant=lambda x: x["site"].astype(str) + x["mutant"])
    .merge(site_numbering_map, on="site", validate="many_to_one")
)

assert func_effects["site_mutant"].nunique() == len(func_effects)

escape

serum,site,wildtype,mutant,sera1-01,sera1-02,sera1-03,sera1-04,sera1-05,sera2-01,sera2-04,sera2-05,site_mutant
0,2,E,A,0.019930,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2A
1,2,E,C,0.043570,-0.007879,0.05043,0.057150,-0.006042,0.07697,0.05997,0.039850,2C
2,2,E,D,NaN,0.288800,0.15600,-0.087910,0.026780,0.26090,0.01403,0.153000,2D
3,2,E,F,-0.158300,0.127500,0.05859,-0.007849,0.046290,0.07214,0.02264,-0.013010,2F
4,2,E,G,0.001388,0.091050,-0.04513,-0.020370,0.005345,-0.07698,0.02925,-0.292700,2G
...,...,...,...,...,...,...,...,...,...,...,...,...
2805,567,I,C,0.286900,0.079450,-0.03988,-0.086600,-0.141700,-0.02005,-0.07882,-0.064730,567C
2806,567,I,G,NaN,-0.252700,-0.19740,-0.083310,NaN,-0.19010,-0.25760,-0.662600,567G
2807,567,I,L,NaN,0.094700,-0.05316,-0.003195,-0.146300,-0.03702,-0.20260,-0.082610,567L
2808,567,I,V,-0.028730,-0.127300,-0.07234,-0.065560,-0.008174,0.07227,-0.16330,-0.002466,567V


Now make a site summary escape plot for all the sera:

In [14]:
floor_escape_at_zero = alt.param(
    value=init_floor_escape_at_zero,
    bind=alt.binding_radio(options=[True, False], name="floor escape at zero"),
)

site_stats = ["mean", "sum", "max", "min"]
site_escape_selection = alt.selection_point(
    fields=["site escape statistic"],
    bind=alt.binding_select(
        options=site_stats,
        name="site escape statistic",
    ),
    value=init_site_escape_stat,
)

site_selection = alt.selection_point(fields=["site"], on="mouseover", empty=False)

func_effects_slider = alt.selection_point(
    fields=["cutoff"],
    value=[{"cutoff": init_min_func_effect}],
    bind=alt.binding_range(
        name="minimum mutation functional effect",
        min=func_effects["functional effect"].min(),
        max=0,
    ),
)

site_brush = alt.selection_interval(
    encodings=["x"],
    mark=alt.BrushConfig(stroke="black", strokeWidth=2, fillOpacity=0),
    empty=False,
)

site_escape_width = 600  # width of site escape chart

site_escape_base = (
    alt.Chart(escape)
    .encode(
        y=alt.Y(
            "escape:Q",
            scale=alt.Scale(nice=False, padding=5),
            axis=alt.Axis(grid=False),
        ),
        tooltip=[
            "site",
            alt.Tooltip("escape:Q", format=".2f"),
            "wildtype",
            "sequential_site:Q",
            "serum:N",
            "region:N",
        ],
    )
)

site_escape_lines = site_escape_base.mark_line(size=0.75, opacity=1)

site_escape_points = site_escape_base.encode(
    strokeWidth=alt.condition(site_selection, alt.value(3), alt.value(0)),
).mark_circle(filled=True, opacity=1, stroke="red", size=20)

site_escape_lines_and_points = (
    (site_escape_lines + site_escape_points)
    .transform_fold(fold=list(sera), as_=["serum", "escape_orig"])
    # floor escape at zero if selected
    .transform_calculate(
        escape=alt.expr.if_(
            floor_escape_at_zero,
            alt.expr.max(alt.datum["escape_orig"], 0),
            alt.datum["escape_orig"],
        )
    )
    # filter on functional effects
    .transform_lookup(
        lookup="site_mutant",
        from_=alt.LookupData(
            func_effects,
            key="site_mutant",
            fields=["functional effect"],
        ),
    )
    .transform_filter(alt.datum["functional effect"] >= func_effects_slider["cutoff"])
    # compute site statistics from mutation statistics
    .transform_aggregate(
        **{stat: f"{stat}(escape)" for stat in site_stats},
        groupby=["site", "serum", "wildtype"],
    )
    # filter on site statistic of interest
    .transform_fold(fold=site_stats, as_=["site escape statistic", "escape"])
    .transform_filter(site_escape_selection)
    # get sequential sites and regions
    .transform_lookup(
        lookup="site",
        from_=alt.LookupData(
            site_numbering_map,
            key="site",
            fields=["sequential_site", "region"],
        ),
    )
)

site_escape = (
    site_escape_lines_and_points
    .encode(
        x=alt.X(
            "site:N",
            sort=alt.SortField("sequential_site:Q"),
            axis=alt.Axis(labelOverlap=True, grid=False),
        ),
        color=alt.condition(site_brush, alt.value("black"), alt.value("darkgray")),
    )
    .properties(height=50, width=site_escape_width)
    .facet(
        facet=alt.Facet(
            "serum:N",
            title="individual sera",
            header=alt.Header(
                labelOrient="right",
                labelFontSize=10,
                labelPadding=3,
                titleOrient="right",
                titlePadding=3,
            ),
        ),
        columns=1,
        spacing=0,
    )
)

site_mean_escape = (
    site_escape_lines_and_points
    # average missing values as zero
    .transform_calculate(
        escape=alt.expr.if_(
            alt.expr.isValid(alt.datum["escape"]),
            alt.datum["escape"],
            0,
        ),
    )
    # take mean over sera
    .transform_aggregate(
        escape="mean(escape)",
        groupby=["site", "wildtype", "sequential_site", "region"],
    )
    .transform_calculate(serum="'mean of all sera'")
    .encode(
        x=alt.X(
            "site:N",
            sort=alt.SortField("sequential_site:Q"),
            axis=None,
        ),
        color=alt.condition(site_brush, alt.value("black"), alt.value("gray")),
    )
    .properties(
        height=80,
        width=site_escape_width,
        title=alt.TitleParams(
            "mean of sera", fontSize=11, fontWeight="bold", orient="right",
        ),
    )
)

region_bar = (
    alt.Chart(site_numbering_map)
    .encode(
        x=alt.X(
            "site:N",
            sort=alt.SortField("sequential_site:Q"),
            axis=None,
        ),
        color=alt.Color(
            "region",
            scale=alt.Scale(domain=site_numbering_map["region"].unique())
        ),
        tooltip=["site", "region", "sequential_site"],
    )
    .mark_rect()
    .properties(width=site_escape_width, height=10)
)       

site_chart = alt.vconcat(
    region_bar, 
    alt.vconcat(site_mean_escape, site_escape, spacing=5).add_params(
        site_escape_selection,
        site_selection,
        func_effects_slider,
        floor_escape_at_zero,
    ),
    spacing=0,
).add_params(site_brush)

site_chart

alt.VConcatChart(...)

Now prepare to plot the heatmaps.
First, create a data frame that has the functional effects and the average escape across sera (averaging mutations missing for a serum as zero for that serum):

In [42]:
heatmap_data = (
    pd.concat(
        [
            escape,
            # add wildtype with zero escape
            (
                escape
                [["site", "wildtype"]]
                .drop_duplicates()
                .assign(mutant=lambda x: x["wildtype"])
            ),
        ],
        ignore_index=True,
    )
    .fillna(0)
    .assign(escape=lambda x: x[list(sera)].mean(axis=1))
    .drop(columns=list(sera))
    .merge(func_effects, validate="one_to_one", how="right")
    .drop(columns="site_mutant")
)

heatmap_data

,site,wildtype,mutant,escape,functional effect,sequential_site,region
0,1,M,A,NaN,-8.421,1,HA1
1,1,M,C,NaN,-8.120,1,HA1
2,1,M,D,NaN,-8.521,1,HA1
3,1,M,F,NaN,-8.177,1,HA1
4,1,M,G,NaN,-7.623,1,HA1
...,...,...,...,...,...,...,...
7427,568,*,R,NaN,-6.692,568,HA2
7428,568,*,T,NaN,-4.667,568,HA2
7429,568,*,V,NaN,-2.432,568,HA2
7430,568,*,W,NaN,-4.873,568,HA2


In [45]:
cell_size = 10  # heatmap cell size

heatmap_base = (
    alt.Chart(heatmap_data)
    # convert null values to NaN so they show as NaN in tooltips rather than as 0.0
    .transform_calculate(
        **{
            col: alt.expr.if_(
                alt.expr.isFinite(alt.datum[col]),
                alt.datum[col],
                alt.expr.NaN,
            )
            for col in ["escape", "functional effect"]
        } 
    )
    .encode(
        x=alt.X(
            "site:N",
            sort=alt.SortField("sequential_site"),
        ),
        y=alt.Y(
            "mutant:N",
            title="amino acid",
            sort=polyclonal.alphabets.biochem_order_aas(func_effects["mutant"].unique()),
        ),
        tooltip=[
            "site",
            "mutant",
            alt.Tooltip("escape", format=".2f"),
            alt.Tooltip("functional effect", format=".2f"),
            "wildtype",
            "sequential_site",
            "region",
        ],
    )
    .properties(width=alt.Step(cell_size), height=alt.Step(cell_size))
)

func_effects_heatmap = (
    heatmap_base
    .encode(color="functional effect")
    .mark_rect()
)

func_effects_heatmap

alt.Chart(...)